In [1]:
suppressPackageStartupMessages(library(tidyverse))

In [2]:
results_dir = '/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/01_data/NicheNet_outputs/treatment_effect_cell_circuit/'

In [3]:
exvivo_dir = '/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/01_data/NicheNet_outputs/ligand_activities_cell_circuit/'
cmp4_dir = '/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/01_data/NicheNet_outputs/ligand_activities_FC+CMP4_FC_cell_circuits/'
nintedanib_dir = '/home/niklas/projects/niche_environments_FIBROSIS/HUMAN_exvivo/01_data/NicheNet_outputs/ligand_activities_FC+Nintedanib_FC_cell_circuits/'

In [4]:
cell_type_labels <- c('Aberrant_Basaloid','ectopic_EC','Myofibroblasts','Pericytes','Profibrotic_Macrophages')

In [5]:
cmp4_all_tabs <- data.frame(matrix(0, 0, 3), stringsAsFactors = F, check.names = F)
colnames(cmp4_all_tabs) <- c('ligand', 'treatment_effect_cmp4', 'receiver')

In [6]:
nintedanib_all_tabs <- data.frame(matrix(0, 0, 4), stringsAsFactors = F, check.names = F)
colnames(nintedanib_all_tabs) <- c('ligand', 'treatment_effect_nintedanib', 'receiver')

In [7]:
## categorical labels of 'treatment_effects_'
# +1 --> induced by cmp4/Nintedanib
# +0 --> no effect by cmp4/Nintedanib
# -1 --> repressed by cmp4/Nintedanib

In [8]:
pearson_thresh = 0.0
top = 50

In [9]:
## loop over EX VIVO receiver cell types
for(ct in cell_type_labels){
    
    ## create table of top ligands for each receiver cell after each treatment
    # FC
    fc_tab <- read.csv(paste0(exvivo_dir, 'nichenet_ligand_act_', ct, '.csv'), stringsAsFactors = F)
    fc_tab <- fc_tab[(fc_tab$pearson > pearson_thresh), c("test_ligand", "pearson")]
    fc_tab <- fc_tab[1:min(top, nrow(fc_tab)), ]
    fc_tab$rank_score <- seq(nrow(fc_tab))
    fc_tab$receiver <- ct
    
    # cmp4
    cmp4_tab <- read.csv(paste0(cmp4_dir, 'nichenet_ligand_act_', ct, '.csv'), stringsAsFactors = F)
    cmp4_tab <- cmp4_tab[(cmp4_tab$pearson > pearson_thresh), c("test_ligand", "pearson")]
    cmp4_tab <- cmp4_tab[1:min(top, nrow(cmp4_tab)), ]
    cmp4_tab$rank_score <- seq(nrow(cmp4_tab))
    cmp4_tab$receiver <- ct
    
    # nintedanib
    nintedanib_tab <- read.csv(paste0(nintedanib_dir, 'nichenet_ligand_act_', ct, '.csv'), stringsAsFactors = F)
    nintedanib_tab <- nintedanib_tab[(nintedanib_tab$pearson > pearson_thresh), c("test_ligand", "pearson")]
    nintedanib_tab <- nintedanib_tab[1:min(top, nrow(nintedanib_tab)), ]
    nintedanib_tab$rank_score <- seq(nrow(nintedanib_tab))
    nintedanib_tab$receiver <- ct
    
    ## identify all ligands modulated by one of the two treatments
    cmp4_ligands <- na.omit(union(fc_tab$test_ligand, cmp4_tab$test_ligand))
    nintedanib_ligands <- na.omit(union(fc_tab$test_ligand, nintedanib_tab$test_ligand))
    
    ## tables with all ligands
    # cmp4
    cmp4_results <- data.frame(matrix(0, length(cmp4_ligands), 2), stringsAsFactors = F, check.names = F)
    colnames(cmp4_results) <- c('ligand', 'treatment_effect_cmp4')
    cmp4_results$ligand <- cmp4_ligands
    
    # nintedanib
    nintedanib_results <- data.frame(matrix(0, length(nintedanib_ligands), 2), stringsAsFactors = F, check.names = F)
    colnames(nintedanib_results) <- c('ligand', 'treatment_effect_nintedanib')
    nintedanib_results$ligand <- nintedanib_ligands
    
    ### cmp4 treatment: determine whether ligand is unique to CMP4 or FC
    for(lig in cmp4_ligands){
        intersect <- intersect(cmp4_tab$test_ligand, fc_tab$test_ligand)
        cmp4_unique <- setdiff(cmp4_tab$test_ligand, fc_tab$test_ligand)
        fc_unique <- setdiff(fc_tab$test_ligand, cmp4_tab$test_ligand)
        
        if(lig %in% cmp4_unique){
            cmp4_results[cmp4_results$ligand == lig, 'treatment_effect_cmp4'] = 1
        }
        
        if(lig %in% fc_unique){
            cmp4_results[cmp4_results$ligand == lig, 'treatment_effect_cmp4'] = -1
        }
        
    }
    
    ## Nintedanib treatment: determine whether ligand is unique to Nintedanib or FC
    for(lig in nintedanib_ligands){
        intersect <- intersect(nintedanib_tab$test_ligand, fc_tab$test_ligand)
        nintedanib_unique <- setdiff(nintedanib_tab$test_ligand, fc_tab$test_ligand)
        fc_unique <- setdiff(fc_tab$test_ligand, nintedanib_tab$test_ligand)
        
        if(lig %in% nintedanib_unique){
            nintedanib_results[nintedanib_results$ligand == lig, 'treatment_effect_nintedanib'] = 1
        }
        
        if(lig %in% fc_unique){
            nintedanib_results[nintedanib_results$ligand == lig, 'treatment_effect_nintedanib'] = -1
        }
        
    }
    
    ## summary tables
    if(dim(cmp4_results)[1] != 0){
        cmp4_results$receiver <- ct
        cmp4_all_tabs <- rbind(cmp4_all_tabs, cmp4_results)
    }
    
    if(dim(nintedanib_results)[1] != 0){
        nintedanib_results$receiver <- ct
        nintedanib_all_tabs <- rbind(nintedanib_all_tabs, nintedanib_results)
    }
    
}

In [10]:
## reshape tables: one row per ligand, separate rows for each receiver cell
cmp4_all_tabs <- reshape(cmp4_all_tabs, idvar = 'ligand', timevar = 'receiver', direction = 'wide')
rownames(cmp4_all_tabs) <- cmp4_all_tabs$ligand
cmp4_all_tabs

,ligand,treatment_effect_cmp4.Aberrant_Basaloid,treatment_effect_cmp4.ectopic_EC,treatment_effect_cmp4.Myofibroblasts,treatment_effect_cmp4.Pericytes,treatment_effect_cmp4.Profibrotic_Macrophages
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PTGS2,PTGS2,0,NA,NA,NA,NA
INHBA,INHBA,-1,-1,-1,-1,-1
CXCL5,CXCL5,0,-1,-1,-1,NA
FBN1,FBN1,-1,-1,0,NA,1
LAMB2,LAMB2,-1,-1,-1,NA,NA
EFNA1,EFNA1,-1,NA,-1,1,1
ITGB2,ITGB2,-1,-1,-1,-1,NA
THBS2,THBS2,-1,NA,NA,1,1
SERPINE1,SERPINE1,-1,NA,0,1,NA


In [11]:
dim(cmp4_all_tabs)

[1] 159   6

In [12]:
## reshape tables: one row per ligand, separate rows for each receiver cell
nintedanib_all_tabs <- reshape(nintedanib_all_tabs, idvar = 'ligand', timevar = 'receiver', direction = 'wide')
rownames(nintedanib_all_tabs) <- nintedanib_all_tabs$ligand
nintedanib_all_tabs

,ligand,treatment_effect_nintedanib.Aberrant_Basaloid,treatment_effect_nintedanib.ectopic_EC,treatment_effect_nintedanib.Myofibroblasts,treatment_effect_nintedanib.Pericytes,treatment_effect_nintedanib.Profibrotic_Macrophages
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PTGS2,PTGS2,-1,NA,NA,NA,NA
INHBA,INHBA,-1,-1,0,-1,0
CXCL5,CXCL5,0,-1,-1,-1,NA
FBN1,FBN1,-1,-1,-1,NA,NA
LAMB2,LAMB2,-1,-1,0,NA,1
EFNA1,EFNA1,-1,NA,0,1,1
ITGB2,ITGB2,-1,-1,-1,-1,NA
THBS2,THBS2,-1,NA,1,1,NA
SERPINE1,SERPINE1,-1,NA,-1,1,NA


In [13]:
dim(nintedanib_all_tabs)

[1] 159   6

In [14]:
## save results
write.csv(cmp4_all_tabs, paste0(results_dir, 'exvivo_MASTER_treatment_cmp4_cell_circuit_top', top, '.csv'))

In [15]:
## save results
write.csv(nintedanib_all_tabs, paste0(results_dir, 'exvivo_MASTER_treatment_nintedanib_cell_circuit_top', top, '.csv'))